In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../../Optimus")

In [ ]:
from dask.distributed import Client
Client(n_workers=None, threads_per_worker=None, processes=False, memory_limit="4GB")
import dask.dataframe as dd
dfd = dd.read_csv("DCIGNP2AYL.txt", header=0, sep=",", dtype={"postal_code": object}, encoding="latin-1")
dfd.head()

In [ ]:
import dask.distributed as distributed
dir(distributed)
None

amazon reviews
social media influecers
editorial review

blogs
podcasts >> discounts
storytelling


In [26]:
from optimus import Optimus
op = Optimus("dask")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LuisA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [24]:
df = op.create.dataframe({"id": [1,2,3,4], 
                          "canal1": ["rrss", "rrss", "fisica", "otro"],
                          "canal2": [None, "otro", "fb", None]
                         })

In [25]:
def expand_category(df, cols="*", regex=None, true_value=True, false_value=False, n=10, output_cols=None):
    
    from optimus.helpers.columns import parse_columns
    
    if regex:
        cols = parse_columns(df, regex, is_regex=True)
    else:
        cols = parse_columns(df, cols)
        
    
    from functools import reduce
    values = []
    for i, col in enumerate(cols):
        values += [v["value"] for v in df.cols.frequency(col, n)["frequency"][col]["values"]]

    for value in set(values):
        mask = reduce(lambda a,b: a | b, [df[col]==value for col in cols])
        if callable(output_cols):
            value = output_cols(value)
        elif output_cols:
            value = f"{output_cols}_{value}"
        df = df.cols.set(value, true_value, default=false_value, where=mask)
        
    return df

expand_category(df, regex="canal", output_cols="canal")

id 1 (int64),canal1 2 (object),canal2 3 (object),canal_rrss 4 (bool),canal_otro 5 (bool),canal_fb 6 (bool),canal_fisica 7 (bool)
1,rrss,None,True,False,False,False
2,rrss,otro,True,True,False,False
3,fisica,fb,False,False,True,True
4,otro,None,False,True,False,False


In [18]:
type(r"a")

str

In [ ]:
dask_client = distributed.Client(n_workers=None, threads_per_worker=None, processes=False, memory_limit="4GB")

In [ ]:
import dask.dataframe as dd
dfd = dd.read_csv("DCIGNP2AYL.txt", header=0, sep=",", dtype={"postal_code": object}, encoding="latin-1", blocksize=1/2)

In [ ]:
dfd

In [38]:
from optimus import Optimus 
op = Optimus("dask", processes=True)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LuisA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [41]:
df = op.load.file("DCIGNP2AYL.txt", n_partitions=2, n_rows=20)
df.data.npartitions

2

In [35]:
df.rows.count()

1631

In [ ]:
df.display()

In [ ]:
df.cols.upper("name").data

In [ ]:
dfd.head(20, npartitions=1, compute=True, safe=False)

In [ ]:
??dfd._head

In [ ]:
dfd.persist()

In [ ]:
dfd = dfd.head(compute=False)

In [ ]:
dfd.partitions[0].compute()

In [ ]:
import numpy as np
for col in df2.data.columns:
    df2.data[col] = dd.from_array(df2.data[col].values)

In [ ]:
df2.data.head()

In [ ]:
dfd.head()

In [ ]:
from optimus.engines.dask.dataframe import DaskDataFrame
df = DaskDataFrame(dfd)

In [ ]:
df.display()

In [ ]:
from dask.distributed import get_client
get_client()

In [ ]:
?dd.read_csv

In [ ]:
import pandas as pd
?pd.read_csv

In [ ]:
import pandas as pd
pd.DataFrame({col: df[col].values for col in df.columns})

In [ ]:
df["a"].compute()

In [ ]:
dir(df.partitions[0])

In [ ]:
import pandas as pd
import numpy as np
pd.Series(np.radians(df.data['latitude'])) - pd.Series(np.radians(df.data['longitude']))

In [ ]:
df['latitude'] - df['longitude']

In [ ]:
from optimus.functions import F

def haversine(lat1, lon1, lat2, lon2):    
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(F.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1         
    a = F.sin(dlat/2)**2 + F.cos(lat1) * F.cos(lat2) * F.sin(dlon/2)**2
    c = 2 * F.asin(F.sqrt(a))  
    total_miles = MILES * c
    return total_miles


In [ ]:
%%time
df["distance"] = haversine(40.671, -73.985, df['latitude'], df['longitude'])

In [ ]:
import numpy as np
np.cos(40.671)